## 全景图原理

In [1]:
import numpy as np
import imutils
import cv2

import sys
sys.path.append('..')
from opencv_course import ImageUtil

## 自己来编写缝合器

![缝合流程](image/StitchingPipeline.jpg)

### warpPerspective计算公式
![warpPerspective计算公式](https://docs.opencv.org/2.4.9/_images/math/4d8aae9979b955d5a94ac070827113377c052a5a.png)

In [2]:
class Stitcher(object):
    pass

In [3]:
def stitch(self, images, ratio=0.75, reprojThresh=4.0,
    showMatches=False):
    # unpack the images, then detect keypoints and extract
    # local invariant descriptors from them
    (imageA, imageB) = images
    (kpsA, featuresA) = self.detectAndDescribe(imageA)
    (kpsB, featuresB) = self.detectAndDescribe(imageB)

    # match features between the two images
    M = self.matchKeypoints(kpsA, kpsB,
        featuresA, featuresB, ratio, reprojThresh)

    # if the match is None, then there aren't enough matched
    # keypoints to create a panorama
    if M is None:
        return None

    # otherwise, apply a perspective warp to stitch the images
    # together
    (matches, H, status) = M
    result = cv2.warpPerspective(imageB, H,
        (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
    result[0:imageA.shape[0], 0:imageA.shape[1]] = imageA

    #计算 图像B的近端x值
    x1 = (H[0][0] * imageB.shape[1] + H[0][2]) /(H[2][0] * imageB.shape[1] + H[2][2])
    x2 = (H[0][0] * imageB.shape[1] + H[0][1]*imageB.shape[0] + H[0][2]) /(H[2][0] * imageB.shape[1] + H[2][1] * imageB.shape[0] + H[2][2])
    x = min(x1, x2)
    result = result[0:imageA.shape[0],0:int(x)]


    # check to see if the keypoint matches should be visualized
    if showMatches:
        vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches, status)

        # return a tuple of the stitched image and the
        # visualization
        return (result, vis)

    # return the stitched image
    return result
Stitcher.stitch = stitch

In [4]:
def detectAndDescribe(self, image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect and extract features from the image
    descriptor = cv2.xfeatures2d.SIFT_create()
    (kps, features) = descriptor.detectAndCompute(image, None)

    # convert the keypoints from KeyPoint objects to NumPy
    # arrays
    kps = np.float32([kp.pt for kp in kps])

    # return a tuple of keypoints and features
    return (kps, features)
Stitcher.detectAndDescribe = detectAndDescribe

In [5]:
def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
    ratio, reprojThresh):
    # compute the raw matches and initialize the list of actual
    # matches

    matcher = cv2.DescriptorMatcher_create("BruteForce")
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    matches = []

    # loop over the raw matches
    for m in rawMatches:
        # ensure the distance is within a certain ratio of each
        # other (i.e. Lowe's ratio test)
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append((m[0].trainIdx, m[0].queryIdx))

    # computing a homography requires at least 4 matches
    if len(matches) > 4:
        # construct the two sets of points
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])

        # compute the homography between the two sets of points
        (H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, reprojThresh)

        # return the matches along with the homograpy matrix
        # and status of each matched point
        return (matches, H, status)

    # otherwise, no homograpy could be computed
    return None
Stitcher.matchKeypoints = matchKeypoints

In [6]:
def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
    # initialize the output visualization image
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # loop over the matches
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        # only process the match if the keypoint was successfully
        # matched
        if s == 1:
            # draw the match
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

    # return the visualization
    return vis
Stitcher.drawMatches = drawMatches


不同版本SIFT create方法不同

        if self.isv3:
            # detect and extract features from the image
            descriptor = cv2.xfeatures2d.SIFT_create()
            (kps, features) = descriptor.detectAndCompute(image, None)

        # otherwise, we are using OpenCV 2.4.X
        else:
            # detect keypoints in the image
            detector = cv2.FeatureDetector_create("SIFT")
            kps = detector.detect(gray)

            # extract features from the image
            extractor = cv2.DescriptorExtractor_create("SIFT")
            (kps, features) = extractor.compute(gray, kps)

In [8]:
!pwd

/home/bbot/opencv_course/aa


In [11]:
import matplotlib.pyplot as plt

imageA = cv2.imread("1.jpg")
imageB = cv2.imread("2.jpg")

# stitch the images together to create a panorama
stitcher = Stitcher()
(result, vis) = stitcher.stitch([imageA, imageB], showMatches=True)

ImageUtil.imshow([imageA, imageB], columns=2, rows=1,figsize=(30, 60))
ImageUtil.imshow([vis], figsize=(50, 40))
ImageUtil.imshow([result], figsize=(50, 40))

[[[ 55  54  59]
  [ 54  53  58]
  [ 55  52  56]
  ...
  [251 251 251]
  [251 251 251]
  [251 251 251]]

 [[ 54  53  56]
  [ 55  54  57]
  [ 55  54  57]
  ...
  [251 251 251]
  [251 251 251]
  [251 251 251]]

 [[ 55  54  57]
  [ 55  54  57]
  [ 57  54  60]
  ...
  [251 251 251]
  [251 251 251]
  [251 251 251]]

 ...

 [[197 189 157]
  [194 186 156]
  [188 179 154]
  ...
  [ 57  58  63]
  [ 57  58  63]
  [ 57  58  63]]

 [[197 188 161]
  [195 186 161]
  [188 179 156]
  ...
  [ 57  58  63]
  [ 57  58  63]
  [ 57  58  63]]

 [[196 188 158]
  [195 186 159]
  [188 179 156]
  ...
  [ 55  56  61]
  [ 57  58  63]
  [ 55  56  61]]]


error: OpenCV(4.1.0) /home/bbot/opencv41/opencv_contrib-4.1.0/modules/xfeatures2d/src/sift.cpp:1207: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'create'
